# Results Formatting

This notebook was used for collecting and formatting the results of the experiments, for the evaluation section of the paper.

### Imports

In [1]:
import pandas as pd
import os
import src.Common.Utils.Config.ConfigHelper as ConfigHelper
import shutil
import src.Common.EpisodeReplay.EpisodeReplay as EpisodeReplay

### Define Constants

In [2]:
RunGroup = "12"
EnvNames = ["FrozenLake"]
BehaviouralTypes = ["Human", "Curated", "HighScore"]


# manual Review of the results config
MaxChoicesPerAgent = 5
MaxReplaysPerChoice = 5


In [3]:
# copy demos of each behavioural type to the run group folder
def CopyDemos(envName, runGroup):
	fromPath = os.path.join("Data", envName, "dev", "replays", "Human")
	toPath = os.path.join("Data", envName, runGroup, "replays", "Human")

	if os.path.exists(toPath):
		shutil.rmtree(toPath)
	shutil.copytree(fromPath, toPath)

	print("Copied Human Demos for: " + envName)
	return

def AddDemoIdsToBehaviour(envName, runGroup, behaviourType):
	# load the results
	statsPath = os.path.join("Data", envName, runGroup, "replays", "Human", "stats.tsv")
	stats = pd.read_csv(statsPath, sep="\t")

	lowerBehaviourType = behaviourType.lower()

	stats["Behaviour"] = stats["loggerName"].apply(lambda x: x.split("_")[-2])

	if lowerBehaviourType != "highscore":
		stats = stats[stats["Behaviour"] == lowerBehaviourType]

	episodeIds = stats["EpisodeId"].unique().tolist()

	loggerName = "Human_" + behaviourType

	# load the json with the episode Ids of the behavioural type
	episodeIdsPath = os.path.join("Data", envName, runGroup, f"{behaviourType}_Episodes.json")
	
	episodeIdsJson = ConfigHelper.LoadConfig(episodeIdsPath)
	episodeIdsJson[loggerName] = episodeIds

	ConfigHelper.SaveConfig(episodeIdsJson, episodeIdsPath)

	print(f"Added {len(episodeIds)} {behaviourType} episodes to {envName}")
	return

for envName in EnvNames:
	CopyDemos(envName, RunGroup)

	for behaviourType in BehaviouralTypes:
		AddDemoIdsToBehaviour(envName, RunGroup, behaviourType)


Copied Human Demos for: FrozenLake
Added 59 Human episodes to FrozenLake
Added 23 Curated episodes to FrozenLake
Added 82 HighScore episodes to FrozenLake


## Collect Replays For Manual Reviewed

In [5]:
def LoadReplay(envName, runGroup, agentType, episodeId):
	
	path = os.path.join("Data", envName, runGroup, "replays", agentType, episodeId)
	try:
		replay = EpisodeReplay.EpisodeReplay.LoadFromFolder(path)
		return replay
	except:
		return None

def CollectReplaysToReview(envName, runGroup, behaviourType):
	episodeIdsPath = os.path.join("Data", envName, runGroup, f"{behaviourType}_Episodes.json")
	replays =  ConfigHelper.LoadConfig(episodeIdsPath)

	columns = ["AgentId", "Predicted", "AgentType"]
	columns += [f"Replay_{i}" for i in range(MaxReplaysPerChoice)]

	replaysToReview = pd.DataFrame(columns=columns)

	for agentId, episodeIds in replays.items():
		
		agentType = agentId.split("_")[0]
		ids = []

		for i in range(len(episodeIds)):
			episodeId = episodeIds[i]

			replay = LoadReplay(envName, runGroup, agentType, episodeId)
			if replay is None:
				continue

			ids.append(episodeId)

			if len(ids) >= MaxReplaysPerChoice or i == len(episodeIds) - 1:
				row = {}
				row["AgentId"] = [agentId]
				row["Predicted"] = [None]
				row["AgentType"] = [agentType]
				for i, id in enumerate(ids):
					row[f"Replay_{i}"] = [id]

				replaysToReview = pd.concat([replaysToReview, pd.DataFrame(row)], ignore_index=True)
				ids = []

	return replaysToReview
	
for envName in EnvNames:
	for behaviourType in BehaviouralTypes:

		if behaviourType == "HighScore":
			continue

		replaysToReview = CollectReplaysToReview(envName, RunGroup, behaviourType)
		replaysToReview = replaysToReview.sample(frac=1)
		replaysToReviewPath = os.path.join("Data", envName, RunGroup, f"ReplaysToReview_{behaviourType}.json")
		replaysToReview.to_json(replaysToReviewPath, orient="records", indent=4)

		print(f"Collected {len(replaysToReview)} replays to review for {behaviourType} in {envName}")


Collected 92 replays to review for Human in FrozenLake
Collected 85 replays to review for Curated in FrozenLake


## Formate the results of the review

In [38]:
behaviourReviewResults = pd.read_json(replaysToReviewPath)
grouped = behaviourReviewResults.groupby("AgentType").aggregate({"Predicted": ["count", "sum"]})


grouped["Percent"] = grouped["Predicted"]["sum"] / grouped["Predicted"]["count"]
grouped["Norm_Percent"] = grouped["Percent"] / grouped["Percent"]["Human"]

# format the percent columns
grouped["Percent"] = grouped["Percent"].apply(lambda x: f"{x:.0%}")
grouped["Norm_Percent"] = grouped["Norm_Percent"].apply(lambda x: f"{x:.0%}")

grouped

Predicted     Percent Norm_Percent
              count sum                     
AgentType                                   
HardCoded        20  20    100%         133%
Human            20  15     75%         100%
ML               40  40    100%         133%
Random           20   0      0%           0%

## Create Tables of agents' results

In [25]:
envs = ["FrozenLake", "CartPole"]
agents = ["Human", "Random", "HardCoded", "ML"]

In [26]:
def LoadStats(envName, runGroup, agent):
	path = os.path.join("Data", envName, runGroup, "replays", agent, "stats.tsv")
	df = pd.read_csv(path, sep="\t")
	return df

In [27]:
df = LoadStats("FrozenLake", "6", "ML")
df

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\FrozenLake\\6\\replays\\ML\\stats.tsv'

In [ ]:
def EnvFitAgentSummary(summaries, df, agentName):
	mean = df["EpisodeTotalReward"].mean()
	sem = df["EpisodeTotalReward"].sem()

	summaries[f"{agentName} Mean"] = mean
	summaries[f"{agentName} sem"] = sem
	summaries[f"{agentName} n"] = len(df)

	return summaries

def GetEnvFitSummary(envName, runGroup):
	
	summaries = {}

	for agent in agents:
		try:
			df = LoadStats(envName, runGroup, agent)

			if agent == "ML":
				realSim = df.loc[df["UseRealSim"] == True]
				summaries = EnvFitAgentSummary(summaries, realSim, f"{agent}(Real Sim)")

				learnedSim = df.loc[df["UseRealSim"] == False]
				summaries = EnvFitAgentSummary(summaries, learnedSim, f"{agent}(Learned Model)")

			else:
				summaries = EnvFitAgentSummary(summaries, df, agent)
		except:
			pass

	# summaryDf = pd.DataFrame(summaries)
	return summaries

def GetEnvsSummary(runGroup):
	summaries = {}

	for env in envs:
		summaries[env] = GetEnvFitSummary(env, runGroup)

	df = pd.DataFrame(summaries)
	df = df.transpose()
	return df

df = GetEnvsSummary("6")
df


,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004036,0.002015,991.0,1.0,0.0,990.0,0.719595,0.018478,592.0,1.0,0.0,2097.0
CartPole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,163.0,NaN,1.0


In [ ]:
df.round(3)

,Random Mean,Random sem,Random n,HardCoded Mean,HardCoded sem,HardCoded n,ML(Real Sim) Mean,ML(Real Sim) sem,ML(Real Sim) n,ML(Learned Model) Mean,ML(Learned Model) sem,ML(Learned Model) n
FrozenLake,0.004,0.002,991.0,1.0,0.0,990.0,0.72,0.018,592.0,1.0,0.0,2097.0
CartPole,nan,nan,nan,nan,nan,nan,nan,nan,0.0,163.0,nan,1.0
